In [1]:
import random

class EightQueens:
    def __init__(self, initial_state=None, mode='empty'):
        self.size = 8
        if initial_state:
            self.state = initial_state #[-1,-1,-1,-1]
        elif mode == 'random':
            self.state = random.sample(range(self.size), self.size)
        else:
            self.state = [-1] * self.size  # -1 means no queen placed
        self.variables = list(range(self.size))  # one variable per row
        self.domains = [list(range(self.size)) for _ in range(self.size)]  # columns for each queen

    def is_consistent(self, row, col):
        """Check if placing a queen at (row, col) does not conflict with previous queens."""
        for r in range(row):
            c = self.state[r]
            if c == col or abs(row - r) == abs(col - c):
                return False
        return True

    def get_actions(self, row):
        """Return all valid columns for placing a queen in the given row."""
        return [col for col in self.domains[row] if self.is_consistent(row, col)]

    def apply_action(self, row, col):
        """Place a queen at the specified position."""
        self.state[row] = col

    def transition_model(self, row, col):
        """Return new state after placing a queen at (row, col)."""
        new_state = self.state[:]
        new_state[row] = col
        return new_state

    def heuristic(self, state = None):
        """Count the number of conflicting queen pairs."""
        conflicts = 0
        if state == None:
            state = self.state
            
        for i in range(self.size):
            for j in range(i + 1, self.size):
                if state[i] == -1 or state[j] == -1:
                    continue
                if state[i] == state[j] or abs(i - j) == abs(state[i] - state[j]):
                    conflicts += 1
        return conflicts
    
    def cost(self, state):
        return sum(1 for col in state if col != -1)

    def print_board(self):
        """Display the board as ASCII."""
        for r in range(self.size):
            row = ["Q" if self.state[r] == c else "." for c in range(self.size)]
            print(" ".join(row))
        print("Heuristic (conflicts):", self.heuristic(self.state))

In [ ]:
from copy import deepcopy, copy
import numpy as np


def inferences(row: int, col: int, domains: list, size: int, defined: list[int]) -> bool:
    for r in range(row + 1, size):
        to_remove = []
        for c in domains[r]:
            if c == col or abs(row - r) == abs(col - c):
                to_remove.append(c)

        for c in to_remove:
            domains[r].remove(c)

        # is column empty and there is no queen in that column
        if not domains[r] and defined[r] == -1:
            return True
        
    return False


def backtrack(csp: EightQueens, row: int,solutions: list, domains: list) -> list:
    
    columns: list = []
    for col in csp.domains[row]:
        columns.append(col)

    for col in columns:
        if csp.is_consistent(row, col):

            # add var = value to assignment
            csp.state[row] = col
            
            # maximum depth reached
            if row == 7 and csp.heuristic() == 0 and csp.cost(csp.state) == 8:
                # return current state, fingers crossed this is correct :D
                solutions[0] += 1
                print()
                print(f'Solution: {solutions}')
                print()
                csp.print_board()
                return csp.state

            # check inference, copy cps.domains as inferences applys changes to it
            domains = deepcopy(csp.domains)
            if not inferences(row, col, csp.domains, csp.size, csp.state):

                # recursive call
                result = backtrack(csp, row + 1, solutions, csp.domains)
        
            # remove var = value and inferences from assignment
            csp.state[row] = -1
            csp.domains = domains
    
    # no solution found on this depth, go up
    # failue would be [-1] * csp.size
    return csp.state

In [ ]:
csp = EightQueens()
solutions = [0]
result = backtrack(csp, 0, solutions, csp.domains)

Q . . . . . . .
. . . . Q . . .
. . . . . . . Q
. . . . . Q . .
. . Q . . . . .
. . . . . . Q .
. Q . . . . . .
. . . Q . . . .
Heuristic (conflicts): 0


('Result state', [0, 4, 7, 5, 2, 6, 1, 3])